## 💰💰💰 Money bags

In [0]:
create or alter proc CreatePyModelRealtimeScoringOnly as
	
	declare @model varbinary(max), @train_script nvarchar(max);
	delete top(1) from models where model_name = 'RevoMMLRealtimeScoring' and language = 'Py';
	
	--The Python script we want to execute
	set @train_script = N'
from microsoftml import rx_logistic_regression, featurize_text, n_gram
from revoscalepy import rx_serialize_model, RxOdbcData, rx_write_object, RxInSqlServer, rx_set_compute_context, RxLocalSeq

connection_string = "Driver=SQL Server;Server=localhost;Database=tpcxbb_1gb;Trusted_Connection=true;"
dest = RxOdbcData(connection_string, table = "models")
 
training_data["tag"] = training_data["tag"].astype("category")

modelpy = rx_logistic_regression(formula = "tag ~ features",
								 data = training_data, 
								 method = "multiClass", 
								 ml_transforms=[featurize_text(language="English",
															   cols=dict(features="pr_review_content"),
															   word_feature_extractor=n_gram(2, weighting="TfIdf"))],
								 train_threads=1)

modelbin = rx_serialize_model(modelpy, realtime_scoring_only = True)
rx_write_object(dest, key_name="model_name", key="RevoMMLRealtimeScoring", value_name="model", value=modelbin, serialize=False, compress=None, overwrite=False)'; --overwrite=false on 2019, true on 2017.

	exec sp_execute_external_script @language = N'Python'
		,@script = @train_script
		,@input_data_1 = N'select * from product_reviews_training_data'
		,@input_data_1_name = N'training_data'
go

## 🐾 Necessary steps
https://docs.microsoft.com/azure/cognitive-services/text-analytics/quickstarts/python, 
https://azure.microsoft.com/services/cognitive-services/text-analytics

In [1]:
import requests
from pprint import pprint
from IPython.display import HTML

subscription_key = "mykey"
text_analytics_base_url = "https://eastus2.api.cognitive.microsoft.com/text/analytics/v2.0/"
sentiment_url = text_analytics_base_url + "sentiment"

documents = {"documents" : [
    {
        "id": 1,
        "language": "en",
        "text": "Works fine. Easy to install. Some reviews talk about not fitting wall plates. Designed for the best, while greet dinner guests, smelling stronger than the Vollarth. While the handle's grip is nice on the OXO Good Grips Trigger Ice Cream Scoop purchased recently and this is the same as all the difference in the kitchen. If you cook for living, go for the professional series."
    }
]}

headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
response  = requests.post(sentiment_url, headers=headers, json=documents)
sentiments = response.json()
pprint(sentiments)

{'documents': [{'id': '1', 'score': 0.9685916900634766}], 'errors': []}


## 🍖 JSON Path

In [1]:
select  top 10
        row_number() over (order by (select 1)) as id,
        'en' as language,
        pr_review_content as text 
from    tpcxbb_1gb..product_reviews_training_data  
for json path, root('documents')

(10 rows affected)

Total execution time: 00:00:00.1093385

JSON_F52E2B61-18A1-11d1-B105-00805F49916B


## 🐱‍🚀 Don't forget the 🔑
[TrollhunterKeys](https://portal.azure.com/#@fleitasarts.com/resource/subscriptions/eaab21d5-8ecd-4ef0-a0c4-92fac2e22875/resourceGroups/rg4/providers/Microsoft.CognitiveServices/accounts/trollhunters/keys)

In [1]:
import requests
from pprint import pprint
from IPython.display import HTML

subscription_key = "mykey" 
text_analytics_base_url = "https://eastus2.api.cognitive.microsoft.com/text/analytics/v2.0/"
sentiment_url = text_analytics_base_url + "sentiment"

documents = {
    "documents": [
        {
            "id": 1,
            "language": "en",
            "text": "Works fine. Easy to install. Some reviews talk about not fitting wall plates. Designed for the best, while greet dinner guests, smelling stronger than the Vollarth. While the handle's grip is nice on the OXO Good Grips Trigger Ice Cream Scoop purchased recently and this is the same as all the difference in the kitchen. If you cook for living, go for the professional series."
        },
        {
            "id": 2,
            "language": "en",
            "text": "great product to save money! Dont worry about leaving the light on anymore. It is great for kitchen! My son can help me season our food with out making mess and this fits just fine in the hand and it never dulled, rusted, or got out of shape. Perfect quality and very easy and effortless to use. This blade is ideal for both narrow and wide wedges. The curve at the local Home Depot store. Both seem to work with. In my case fan). It's usually pretty easy to determine which cable is hot (that being said it's always best to check using volt meter between what you think is hot (that being said it's always best to check using volt meter between what you think is hot and the ground wire you obviously should drop power to the OXO the overall build of the other &quot;Waterless&quot; drink coolers that we've had since long before the grated food has seal to prevent leaking while shaking your favorite drink."
        },
        {
            "id": 3,
            "language": "en",
            "text": "Next time will go with the old metal handle- this is bonus."
        },
        {
            "id": 4,
            "language": "en",
            "text": "Great Gift Great Value had to get used. And after 12 hours of use, they just throw them away, so you haven't created any useless clutter. (Get yourself set too.)"
        },
        {
            "id": 5,
            "language": "en",
            "text": "After trip to Paris and falling in love with Nutella crepes decided had to try it. am glad found it! Thank you, CIA, for my existing switch. Design-wise it is dishwasher safe too! Very highly recommended. You'll thank me for this!JANA"
        },
        {
            "id": 6,
            "language": "en",
            "text": "Simply the best thing about them is that you can only use for one thing, so this one is wonderful to hold the keys."
        },
        {
            "id": 7,
            "language": "en",
            "text": "This is the exact product that my mother used in the outlet/switch box. It does exactly what was glad to find so was happy to finally get them. great service. thank you."
        },
        {
            "id": 8,
            "language": "en",
            "text": "Not super magnet, but strong enough to set on the oven and the spatula is supposed to have, but this one is definitely heavy duty! have placed 15 minute timer on all the time and will certainly provide entertainment for your guests. (It is such great gift in festival's sovenuior such as this to get used. And after 12 hours of use, they just throw them away, so you haven't created any useless clutter. (Get yourself set too.)"
        },
        {
            "id": 9,
            "language": "en",
            "text": "Installed as bathroom fan timer. Easy to install. Some reviews talk about not fitting wall plates. Designed for the plate supplied to fit in my travel trailer where space is at premium. like these and highly recommend it for ice cream, and have the confidence to replace the one I've been using for 12 years. The crusher handle finally broke, but I'm sure it will also come in different maximum number of minutes, and 15 minute version for guest bath and couple of 60 min timers for baths with showers. Installed quiet fans and we can turn on the metal trigger.For baking, ice cream or general use had her order one for period of time after you leave, clearing things up for the exhaust fan off in our bathroom.Saves money on heat and cooling..."
        },
        {
            "id": 10,
            "language": "en",
            "text": "Our home was built in 2003 and this fits just fine in the drawer until find one of those things that if was looking for, good quality, and after months of daily service.."
        }
    ]
}
print(type(documents))

headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
response  = requests.post(sentiment_url, headers=headers, json=documents)
sentiments = response.json()
pprint(sentiments)

<class 'dict'>


{'documents': [{'id': '1', 'score': 0.9685916900634766},
               {'id': '2', 'score': 0.874512791633606},
               {'id': '3', 'score': 0.7775521278381348},
               {'id': '4', 'score': 0.1461590826511383},
               {'id': '5', 'score': 0.9813788533210754},
               {'id': '6', 'score': 0.8957217931747437},
               {'id': '7', 'score': 0.9916195869445801},
               {'id': '8', 'score': 0.08493909239768982},
               {'id': '9', 'score': 0.8297852277755737},
               {'id': '10', 'score': 0.7934412956237793}],
 'errors': []}


## 🧠‍‍ 👓

In [1]:
drop view if exists JsonDocuments
go
create view JsonDocuments
as 
select (
    select  top 10
            row_number() over (order by (select 1)) as id, 
            'en' as language, 
            pr_review_content as text 
    from    product_reviews_training_data 
    for json path, root('documents')
) as documents
go
exec sp_execute_external_script @language = N'Python'
    ,@script = N'DocOut = DocIn'
    ,@input_data_1 = N'select * from JsonDocuments;'
    ,@input_data_1_name  = N'DocIn'
    ,@output_data_1_name =  N'DocOut'
with result sets ((DocOut varchar(max)));
go

Commands completed successfully.

Total execution time: 00:00:00.0447550

Commands completed successfully.

Total execution time: 00:00:00.0030671

(1 row affected)

Total execution time: 00:00:04.5605194

DocOut


## 💀💀💀💀 died... 🤯 
https://github.com/microsoft/azuredatastudio/issues/5722

{'documents': [{'id': '1', 'score': 0.9685916900634766},
               {'id': '2', 'score': 0.874512791633606},
               {'id': '3', 'score': 0.7775521278381348},
               {'id': '4', 'score': 0.1461590826511383},
               {'id': '5', 'score': 0.9813788533210754},
               {'id': '6', 'score': 0.8957217931747437},
               {'id': '7', 'score': 0.9916195869445801},
               {'id': '8', 'score': 0.08493909239768982},
               {'id': '9', 'score': 0.8297852277755737},
               {'id': '10', 'score': 0.7934412956237793}],
 'errors': []}


## 🐱‍🐉 upgrade with magics
https://pypi.org/simple/urllib3/

## 🐱‍💻 so hard... 🎯
https://stackoverflow.com/questions/47954324/how-to-output-values-from-sp-execute-external-script-into-table 
https://stackoverflow.com/questions/44802160/convert-json-api-response-to-pandas-dataframe

In [2]:
declare @py nvarchar(max);
	
set @py = N'from pandas.io.json import json_normalize

rds = {"documents": [{"id": "1", "score": 0.97},{"id": "2","score": 0.87},{"id": "3", "score": 0.78}],"errors": []}
print(type(rdd)) #dict

df = json_normalize(rds, "documents")
print(df)
print(type(df)) #dataframe
'; 

drop table if exists apiresults
create table apiresults (id int, score float)

insert into apiresults
exec sp_execute_external_script 
	@language = N'Python',
	@script = @py,
	@output_data_1_name =N'df'
	
select * from apiresults

STDOUT message(s) from external script: 
 
 id score
0 1 0.97
1 2 0.87
2 3 0.78

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:04.2972241

id,score
1,0.97
2,0.87
3,0.78


## 🐱‍👤 Gato Ninja, Don't forget the 🔑
https://repl.it/languages/python3

In [10]:
create or alter proc GetCognitiveAPI 
as
	declare @py nvarchar(max);
	
	set @py = N'import requests
from pandas.io.json import json_normalize

subscription_key = "mykey" 
text_analytics_base_url = "https://eastus2.api.cognitive.microsoft.com/text/analytics/v2.0/"
sentiment_url = text_analytics_base_url + "sentiment"

df = jsondocs

headers  = {"Ocp-Apim-Subscription-Key": subscription_key, "content-type": "application/json"}
response = requests.post(sentiment_url, headers = headers, data = df.iloc[0][0].encode()) 

rds = response.json()
df2 = json_normalize(rds, "documents")'; 

	drop table if exists apiresults;
	create table apiresults (id int, score float);

	insert into apiresults
	exec sp_execute_external_script @language = N'Python'
		,@script = @py
		,@input_data_1 = N'select * from JsonDocuments'
		,@input_data_1_name = N'jsondocs'
   		,@output_data_1_name =  N'df2'
	select * from apiresults;
go
exec GetCognitiveAPI
go

Commands completed successfully.

Total execution time: 00:00:00.0077552

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:05.0466495

id,score
1,0.968591690063477
2,0.874512791633606
3,0.777552127838135
4,0.146159082651138
5,0.981378853321075
6,0.895721793174744
7,0.99161958694458
8,0.0849390923976898
9,0.829785227775574
10,0.793441295623779
